In [7]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Get current working directory instead of __file__
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from efficientNetModel import EfficientNetWheatModel
from dataLoaderFunc import loadSplitData, createLoader

In [8]:
# ✅ Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for batch_idx, (rgb_batch, dsm_batch, label_batch) in enumerate(train_loader):
            rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)

            optimizer.zero_grad()
            outputs = model(rgb_batch, dsm_batch)
            loss = criterion(outputs, label_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            if batch_idx % 20 == 0:
                print(f"Epoch {epoch+1}/{num_epochs} | Batch {batch_idx}/{len(train_loader)} | Loss: {loss.item():.4f}")

        train_loss /= len(train_loader)

        # ✅ Compute Validation Loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for rgb_batch, dsm_batch, label_batch in val_loader:
                rgb_batch, dsm_batch, label_batch = rgb_batch.to(device), dsm_batch.to(device), label_batch.to(device)
                outputs = model(rgb_batch, dsm_batch)
                loss = criterion(outputs, label_batch)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        scheduler.step(val_loss)

        print(f"✅ Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # ✅ Save Best Model
        torch.save(model.state_dict(), "efficientnet_wheat_model.pth")

# ✅ Test the model on validation set
def test_model(model, test_loader):
    model.eval()
    predictions, actuals = [], []

    with torch.no_grad():
        for rgb_batch, dsm_batch, label_batch in test_loader:
            rgb_batch, dsm_batch = rgb_batch.to(device), dsm_batch.to(device)
            outputs = model(rgb_batch, dsm_batch)
            predictions.extend(outputs.cpu().numpy().flatten())
            actuals.extend(label_batch.cpu().numpy().flatten())

    return predictions, actuals


In [9]:
train_df, val_df, test_df = loadSplitData("RGB_DSM_totEarNum.csv")
train_loader, val_loader, test_loader = createLoader(train_df, val_df, test_df)

Train Size: 47840, Validation Size: 5980, Test Size: 5980
Train Batches: 2990, Validation Batches: 374, Test Batches: 374


In [11]:

# ✅ Initialize Model, Loss Function, and Optimizer

# ✅ Universal device selection (works on both Mac M2 & Windows with RTX 4060)
if torch.backends.mps.is_available():
    device = "mps"  # ✅ Use Apple Metal (Mac M1/M2)
    torch.set_default_tensor_type(torch.FloatTensor)
elif torch.cuda.is_available():
    device = "cuda"  # ✅ Use NVIDIA CUDA (Windows RTX 4060)
else:
    device = "cpu"  # ✅ Default to CPU if no GPU is available
print(f"✅ Using device: {device}")

model = EfficientNetWheatModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Lower learning rate for EfficientNet
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)
# ✅ Start Training

✅ Using device: cuda


In [12]:
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs=10)

Epoch 1/10 | Batch 0/2990 | Loss: 138205.1250
Epoch 1/10 | Batch 20/2990 | Loss: 72352.3047
Epoch 1/10 | Batch 40/2990 | Loss: 13863.5352
Epoch 1/10 | Batch 60/2990 | Loss: 6886.5967
Epoch 1/10 | Batch 80/2990 | Loss: 4209.9443
Epoch 1/10 | Batch 100/2990 | Loss: 3245.1499
Epoch 1/10 | Batch 120/2990 | Loss: 2727.2837
Epoch 1/10 | Batch 140/2990 | Loss: 3461.9041
Epoch 1/10 | Batch 160/2990 | Loss: 4402.2686
Epoch 1/10 | Batch 180/2990 | Loss: 7920.2261
Epoch 1/10 | Batch 200/2990 | Loss: 7893.5557
Epoch 1/10 | Batch 220/2990 | Loss: 7634.8281
Epoch 1/10 | Batch 240/2990 | Loss: 4557.2275
Epoch 1/10 | Batch 260/2990 | Loss: 7128.0522
Epoch 1/10 | Batch 280/2990 | Loss: 4141.8965
Epoch 1/10 | Batch 300/2990 | Loss: 2541.5493
Epoch 1/10 | Batch 320/2990 | Loss: 6190.6250
Epoch 1/10 | Batch 340/2990 | Loss: 5646.0049
Epoch 1/10 | Batch 360/2990 | Loss: 5274.7295
Epoch 1/10 | Batch 380/2990 | Loss: 4305.2114
Epoch 1/10 | Batch 400/2990 | Loss: 3916.4575
Epoch 1/10 | Batch 420/2990 | Loss: 

In [ ]:
# ✅ Load the saved model
model.load_state_dict(torch.load("efficientnet_wheat_model.pth"))
model.eval()

# Run test
preds, actuals = test_model(model, test_loader)

# ✅ Print sample predictions
for p, a in zip(preds[:10], actuals[:10]):
    print(f"Predicted: {p:.2f}, Actual: {a:.2f}")
